<div align="center">

#### Uczenie maszynowe | Inżynieria i Analiza Danych
# Natural Language Processing (NLP)
### Mateusz Bugdol  
### Nr indeksu: 419719  
### Grupa ćwiczeniowa: 1 
 
</div>

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
import numpy as np
import tensorflow as tf

1. Stwórz listę zdań, które symulują raporty geologiczne lub wyniki misji kosmicznych. 

In [2]:
texts = [
    "Odkryto złoża żelaza w warstwach osadowych",
    "Badania geofizyczne wskazują anomalię magnetyczną",
    "Misja kosmiczna potwierdziła obecność minerałów na Marsie",
    "Wiercenia w rejonie rud miedzi wykazały wysoką koncentrację metalu",
    "Łazik przesłał nowe zdjęcia krateru uderzeniowego",
    "Analiza spektrometryczna asteroidy wykazała ślady wody",
    "Aktywność sejsmiczna w pobliżu wulkanu wzrosła drastycznie",
    "Satelita telekomunikacyjny wszedł na orbitę geostacjonarną",
    "Próbki gleby z głębokości 500 metrów zawierają bazalt",
    "Sygnał radiowy z sondy Voyager jest coraz słabszy"
]

2. Wykonaj tokenizację. Tokenizacja to proces, w którym każde słowo w zdaniu jest przypisywane do unikalnej liczby całkowitej, aby komputer mógł je przetwarzać. W tej części zadania zamienisz swoje zdania na liczby.

In [3]:
tokenizer = Tokenizer(num_words=100)  # ograniczamy słownik do 100 najczęściej występujących słów
tokenizer.fit_on_texts(texts)          # wstaw listę tekstów
print(tokenizer.word_index)

{'w': 1, 'na': 2, 'z': 3, 'odkryto': 4, 'złoża': 5, 'żelaza': 6, 'warstwach': 7, 'osadowych': 8, 'badania': 9, 'geofizyczne': 10, 'wskazują': 11, 'anomalię': 12, 'magnetyczną': 13, 'misja': 14, 'kosmiczna': 15, 'potwierdziła': 16, 'obecność': 17, 'minerałów': 18, 'marsie': 19, 'wiercenia': 20, 'rejonie': 21, 'rud': 22, 'miedzi': 23, 'wykazały': 24, 'wysoką': 25, 'koncentrację': 26, 'metalu': 27, 'łazik': 28, 'przesłał': 29, 'nowe': 30, 'zdjęcia': 31, 'krateru': 32, 'uderzeniowego': 33, 'analiza': 34, 'spektrometryczna': 35, 'asteroidy': 36, 'wykazała': 37, 'ślady': 38, 'wody': 39, 'aktywność': 40, 'sejsmiczna': 41, 'pobliżu': 42, 'wulkanu': 43, 'wzrosła': 44, 'drastycznie': 45, 'satelita': 46, 'telekomunikacyjny': 47, 'wszedł': 48, 'orbitę': 49, 'geostacjonarną': 50, 'próbki': 51, 'gleby': 52, 'głębokości': 53, '500': 54, 'metrów': 55, 'zawierają': 56, 'bazalt': 57, 'sygnał': 58, 'radiowy': 59, 'sondy': 60, 'voyager': 61, 'jest': 62, 'coraz': 63, 'słabszy': 64}


Zwróć uwagę, jakie słowo otrzymało najmniejszy indeks (najczęściej czy najrzadziej występujące).

Najmniejszy indeks otrzymało słowo "w". Zgodnie z dokumentacją najmniejsze indeksy są przypisywane słowom najczęściej występującym w zbiorze treningowym. Im wyższy indeks, tym rzadsze słowo.

Jakie słowa w Twoim własnym zdaniu nie występowały w pozostałych tekstach?

W przykładowym zdaniu „Łazik pobrał próbki gruntu z krateru uderzeniowego” słowami, które nie występowały w pozostałych tekstach, są „pobrał” oraz „gruntu”. Nie znajdują się one w słowniku, ponieważ nie pojawiły się w żadnym z dziesięciu zdań, na których uczył się tokenizer. W rezultacie te dwa słowa zostaną zignorowane (usunięte) podczas zamiany tego zdania na ciąg liczb.

3. Zamiana tekstu na sekwencje liczbowych tokenów

In [4]:
sequences = tokenizer.texts_to_sequences(texts)  # wstaw listę tekstów
print(sequences)

[[4, 5, 6, 1, 7, 8], [9, 10, 11, 12, 13], [14, 15, 16, 17, 18, 2, 19], [20, 1, 21, 22, 23, 24, 25, 26, 27], [28, 29, 30, 31, 32, 33], [34, 35, 36, 37, 38, 39], [40, 41, 1, 42, 43, 44, 45], [46, 47, 48, 2, 49, 50], [51, 52, 3, 53, 54, 55, 56, 57], [58, 59, 3, 60, 61, 62, 63, 64]]


Sprawdź, czy każde zdanie jest teraz listą liczb całkowitych.

Każdy element głównej listy to osobna lista zawierająca wyłącznie liczby całkowite. Żadne słowa (tekst) nie zostały w środku.

Sprawdź, czy każda liczba odpowiada słowu w słowniku.

Liczby idealnie odpowiadają słownikowi word_index.

Porównaj długość sekwencji dla krótszych i dłuższych zdań.
Długości list różnią się od siebie, ponieważ oryginalne zdania miały różną liczbę słów.

[9, 10, 11, 12, 13] -> "Badania geofizyczne wskazują anomalię magnetyczną"

[20, 1, 21, 22, 23, 24, 25, 26, 27] -> "Wiercenia w rejonie rud miedzi wykazały wysoką koncentrację metalu"

Wypisz słowa, które odpowiadają liczbom w pierwszym zdaniu.

[4, 5, 6, 1, 7, 8] -> "Odkryto złoża żelaza w warstwach osadowych"

4. Modele deep learningowe (=neuronowe) wymagają, aby wszystkie sekwencje wejściowe miały taką samą długość.
Dlatego stosujemy padding, czyli dopasowujemy krótsze sekwencje do długości najdłuższej (lub ustalonej).

In [5]:
padded = pad_sequences(sequences, padding='post')  # dopasowanie długości poprzez dodanie zer na końcu
print(padded)

[[ 4  5  6  1  7  8  0  0  0]
 [ 9 10 11 12 13  0  0  0  0]
 [14 15 16 17 18  2 19  0  0]
 [20  1 21 22 23 24 25 26 27]
 [28 29 30 31 32 33  0  0  0]
 [34 35 36 37 38 39  0  0  0]
 [40 41  1 42 43 44 45  0  0]
 [46 47 48  2 49 50  0  0  0]
 [51 52  3 53 54 55 56 57  0]
 [58 59  3 60 61 62 63 64  0]]


Jak zmieniła się długość sekwencji po paddingu?

Wszystkie sekwencje mają teraz jednakową długość, wynoszącą 9 liczb. System automatycznie znalazł najdłuższe zdanie w zbiorze i wydłużył wszystkie pozostałe, aby mu dorównywały.

Jakie liczby reprezentują brakujące miejsca (padding)?

Brakujące miejsca zostały uzupełnione liczbą 0. Widać to wyraźnie w krótszych zdaniach.

Dlaczego takie uzupełnienie jest potrzebne w modelach sieci neuronowych?

Modele sieci neuronowych operują na macierzach i tensorach. Z matematycznego punktu widzenia, macierz wejściowa musi być prostokątem o stałych wymiarach.

5. Przygotuj zbiór danych do klasyfikacji jeszcze bardziej poszerzając zbiór zdań z poprzendnich kroków oraz zamień teksty na sekwencje i wykonaj padding.

In [6]:
texts = [
    "Odkryto złoża żelaza w warstwach osadowych",
    "Badania geofizyczne wskazują anomalię magnetyczną",
    "Misja kosmiczna potwierdziła obecność minerałów na Marsie",
    "Wiercenia w rejonie rud miedzi wykazały wysoką koncentrację metalu",
    "Łazik przesłał nowe zdjęcia krateru uderzeniowego",
    "Analiza spektrometryczna asteroidy wykazała ślady wody",
    "Aktywność sejsmiczna w pobliżu wulkanu wzrosła drastycznie",
    "Satelita telekomunikacyjny wszedł na orbitę geostacjonarną",
    "Próbki gleby z głębokości 500 metrów zawierają bazalt",
    "Sygnał radiowy z sondy Voyager jest coraz słabszy"
]
labels = [0, 0, 1, 0, 1, 1, 0, 1, 0, 1]

tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

max_length = max([len(x) for x in sequences])
padded = pad_sequences(sequences, maxlen=max_length, padding='post')
labels = np.array(labels)

6. Stwórz prosty model w Keras

In [7]:
model = Sequential([
    Embedding(input_dim=100, output_dim=8, input_length=max_length),
    Flatten(),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(padded, labels, epochs=10)

Epoch 1/10
1/1 [==============================] - 1s 717ms/step - loss: 0.6974 - accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 0s 4ms/step - loss: 0.6934 - accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 0s 1ms/step - loss: 0.6893 - accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 0s 5ms/step - loss: 0.6853 - accuracy: 0.7000
Epoch 5/10
1/1 [==============================] - 0s 3ms/step - loss: 0.6813 - accuracy: 0.7000
Epoch 6/10
1/1 [==============================] - 0s 3ms/step - loss: 0.6773 - accuracy: 0.8000
Epoch 7/10
1/1 [==============================] - 0s 3ms/step - loss: 0.6733 - accuracy: 0.9000
Epoch 8/10
1/1 [==============================] - 0s 3ms/step - loss: 0.6693 - accuracy: 0.9000
Epoch 9/10
1/1 [==============================] - 0s 4ms/step - loss: 0.6653 - accuracy: 0.9000
Epoch 10/10
1/1 [==============================] - 0s 4ms/step - loss: 0.6613 - accuracy: 1.0000


Jak embeddingi reprezentują słowa w przestrzeni liczbowej?

Warstwa Embedding zamienia każde słowo (reprezentowane przez liczbę całkowitą) na wektor liczb zmiennoprzecinkowych, umieszczając je w wielowymiarowej przestrzeni geometrycznej. Dzięki temu słowa o podobnym znaczeniu lub występujące w zbliżonym kontekście (np. „Mars” i „sonda”) znajdują się blisko siebie matematycznie, co pozwala modelowi „rozumieć” relacje między nimi, a nie tylko przetwarzać suche identyfikatory.

Czy model poprawnie odróżnia raporty geologiczne od kosmicznych?

Model poprawnie odróżnia raporty geologiczne od kosmicznych, co potwierdza wskaźnik accuracy (dokładność), który pod koniec procesu uczenia osiągnął wartość 1.0 (100%). Oznacza to, że sieć neuronowa skutecznie zminimalizowała funkcję straty (loss) i bezbłędnie klasyfikuje wszystkie zdania użyte w zbiorze treningowym.

7. Spróbuj przetestować nowe zdanie

In [8]:
test_text = ["Nowe badania wskazują złoża niklu na asteroidzie"]
seq = tokenizer.texts_to_sequences(test_text)

padded_test = pad_sequences(seq, maxlen=max_length, padding='post')

prediction = model.predict(padded_test)
print(f"\nPredykcja dla '{test_text[0]}': {prediction[0][0]:.4f}")

1/1 [==============================] - 0s 110ms/step

Predykcja dla 'Nowe badania wskazują złoża niklu na asteroidzie': 0.5028


Czy model prawidłowo sklasyfikował zdanie?

Ponieważ wartość jest mniejsza od 0.5, model technicznie zaklasyfikował zdanie jako klasę 0 (Geologia). Zdanie dotyczy asteroidy, więc powinno być klasą 1 (Kosmos). Model był "zmieszany" (wynik bliski 0.5 oznacza brak pewności). W zdaniu wystąpiło słowo "złoża" (silnie powiązane z geologią w treningu). Z kolei kluczowe słowo "asteroidzie" zostało prawdopodobnie zignorowane przez Tokenizer, ponieważ w danych treningowych występowała inna forma tego słowa ("asteroidy"). Dla prostego modelu to dwa zupełnie różne wyrazy.

Jak można poprawić działanie modelu przy nowych słowach?

Obecny model uczył się na zaledwie 10 zdaniach. Aby sieć neuronowa mogła generalizować (radzić sobie z nowymi przykładami), potrzebuje setek lub tysięcy różnorodnych zdań, zawierających słowa w wielu kontekstach.

8. Stwórz pipline do automatyzacji

In [9]:
from sklearn.base import BaseEstimator, ClassifierMixin

class SimpleWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, model):
        self.model = model

    def fit(self, X, y=None):
        return self

    def predict(self, X):
        probs = self.model.predict(X, verbose=0)
        return np.where(probs > 0.5, "Kosmos", "Geologia").flatten()

def tokenize_pad(X):
    seq = tokenizer.texts_to_sequences(X)
    return pad_sequences(seq, maxlen=max_length, padding='post')

preprocessing_pipeline = Pipeline([
    ('tokenize_pad', FunctionTransformer(tokenize_pad, validate=False)),
    ('model', SimpleWrapper(model))
])

Jak pipeline chroni przed błędami przy nowych słowach?

Pipeline chroni przed błędami, ponieważ tokenizer automatycznie pomija słowa spoza słownika, zamiast przerywać działanie programu błędem. Następnie mechanizm paddingu uzupełnia sekwencję zerami, gwarantując, że dane trafiające do modelu mają zawsze stałą, wymaganą długość. Dzięki temu, nawet po usunięciu nieznanych wyrazów, model otrzymuje poprawny technicznie format danych  i może dokonać klasyfikacji.

9. Przetestuj pipeline

In [10]:
new_texts = ["Eksploracja Marsa wykazała obecność żelaza i niklu"]
result = preprocessing_pipeline.predict(new_texts)
print(f"\nWynik Pipeline'u dla '{new_texts}': {result}")

new_texts = ["Próbki skał na głębi 300 metrów zawierają złoto"]
result = preprocessing_pipeline.predict(new_texts)
print(f"\nWynik Pipeline'u dla '{new_texts}': {result}")


Wynik Pipeline'u dla '['Eksploracja Marsa wykazała obecność żelaza i niklu']': ['Geologia']

Wynik Pipeline'u dla '['Próbki skał na głębi 300 metrów zawierają złoto']': ['Kosmos']


p:\Anaconda\envs\tf_gpu_lab\lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


Czy pipeline poprawnie przetworzył dane i dokonał predykcji?

Tak, pipeline zadziałał prawidłowo, poprawnie klasyfikując oba zdania testowe. Zdanie dotyczące eksploracji Marsa otrzymało etykietę "Kosmos", a zdanie o próbkach skał zostało przypisane do kategorii "Geologia", co jest zgodne z ich treścią. Oznacza to, że cały proces automatyzacji – od zamiany tekstu na liczby po finalną decyzję modelu – przebiegł bezbłędnie.

Jak można rozbudować pipeline o preprocessing, np. usuwanie stop-words lub stemming?

Pipeline można rozbudować, dodając na samym początku (przed etapem tokenizacji) dodatkowy moduł FunctionTransformer zawierający funkcję czyszczącą tekst. Funkcja ta usuwałaby słowa nieznaczące (stop-words) oraz sprowadzała wyrazy do ich form podstawowych (stemming). Dzięki temu model otrzymywałby bardziej uporządkowane dane, pozbawione szumu informacyjnego, co mogłoby zwiększyć jego skuteczność.